In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE

In [2]:
train_df = pd.read_csv('./data/trainingset.csv')
test_df = pd.read_csv('./data/testset.csv')
train_df.head()

C:\Users\justi\Anaconda3\envs\deeplearning\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (50,172,255,256,257,258,268,280,376) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\justi\Anaconda3\envs\deeplearning\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (50,255,256,257,258,260,268) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,cntryid,cntryid_e,age_r,gender_r,computerexperience,nativespeaker,edlevel3,monthlyincpr,yearlyincpr,lng_home,...,v85,v50,v89,v127,v239,v224,v71,v105,row,uni
0,Russian Federation,Russian Federation,42.0,Female,Yes,Yes,High,25 to less than 50,25 to less than 50,999,...,Agree,Agree,NaN,NaN,233.0,9996.0,8521,9996.0,95754,jw3580
1,United States,United States,NaN,Male,Yes,Yes,High,75 to less than 90,75 to less than 90,999,...,Agree,Neither agree nor disagree,NaN,NaN,9999.0,9996.0,9999,9996.0,57367,jw3580
2,United States,United States,NaN,Female,Yes,Yes,High,NaN,NaN,999,...,Strongly agree,Strongly agree,NaN,NaN,9996.0,9999.0,9996,9999.0,195554,jw3580
3,Czech Republic,Czech Republic,23.0,Female,Yes,Yes,Medium,10 to less than 25,10 to less than 25,ces,...,Agree,Agree,NaN,NaN,3311.0,9996.0,6419,9996.0,42148,jw3580
4,Canada,Canada (English),NaN,Male,Yes,Yes,Medium,NaN,NaN,999,...,Disagree,Agree,NaN,NaN,9996.0,9999.0,9999,9999.0,86076,jw3580


# Sampling with Replacement for Gender Bias Reduction

In [3]:
vc_train = train_df['gender_r'].value_counts()
vc_train

Male      12495
Female     7505
Name: gender_r, dtype: int64

In [4]:
vc_test = test_df['gender_r'].value_counts()
vc_test

Female    10111
Male       9889
Name: gender_r, dtype: int64

In [5]:
len(test_df)

20000

In [6]:
# find weights
w_m = (vc_test[1] / len(test_df)) / (vc_train[0] / len(train_df))
w_f = (vc_test[0] / len(test_df)) / (vc_train[1] / len(train_df))
w_m, w_f

(0.791436574629852, 1.3472351765489676)

In [7]:
# Probability of drawing
sum_of_w = w_m + w_f
P_m = w_m / sum_of_w
P_f = w_f / sum_of_w
P_m, P_f

(0.37005986271320884, 0.6299401372867911)

In [8]:
# Build up a weights series
weights = train_df['gender_r'] == 'Male'
weights.replace(to_replace=True, value=P_m, inplace=True)
weights.replace(to_replace=False, value=P_f, inplace=True)
weights.shape

(20000,)

In [9]:
train_df = train_df.sample(n=20000, replace=True, weights=weights,
                           random_state=42)
train_df.head()

,cntryid,cntryid_e,age_r,gender_r,computerexperience,nativespeaker,edlevel3,monthlyincpr,yearlyincpr,lng_home,...,v85,v50,v89,v127,v239,v224,v71,v105,row,uni
7502,Russian Federation,Russian Federation,28.0,Female,Yes,Yes,High,25 to less than 50,25 to less than 50,999,...,Agree,Agree,NaN,NaN,1222.0,9996.0,731,9996.0,93061,jw3580
19002,Japan,Japan,35.0,Male,Yes,Yes,High,50 to less than 75,50 to less than 75,jpn,...,Strongly agree,Agree,NaN,NaN,4313.0,9996.0,7820,9996.0,7994,jw3580
14614,Italy,Italy,37.0,Male,Yes,Yes,Medium,75 to less than 90,75 to less than 90,999,...,Neither agree nor disagree,Agree,NaN,NaN,7543.0,9996.0,3020,9996.0,163582,jw3580
11968,United States,United States,NaN,Male,Yes,Yes,High,50 to less than 75,50 to less than 75,999,...,Agree,Agree,NaN,NaN,9999.0,9996.0,9999,9996.0,156617,jw3580
3130,Russian Federation,Russian Federation,38.0,Female,Yes,Yes,High,Less than 10,Less than 10,999,...,Strongly agree,Strongly agree,NaN,NaN,1345.0,9996.0,851,9996.0,53585,jw3580


In [10]:
# Gender bias in sample removed
train_df['gender_r'].value_counts()

Female    10141
Male       9859
Name: gender_r, dtype: int64

In [11]:
# Dropping uni and v262 because the two columns have no infomration
train_df.drop(['uni', 'v262'], axis=1, inplace=True)
test_df.drop(['uni', 'v262'], axis=1, inplace=True)

# NaN processing with interpolation
train_df.fillna(train_df.mean(), inplace=True)
test_df.fillna(test_df.mean(), inplace=True)

# Pick out categorical columns
train_categ_df = train_df.select_dtypes(include=[np.object], exclude=[np.number])
train_df.drop(train_categ_df.columns, axis=1, inplace=True)
test_categ_df = test_df.select_dtypes(include=[np.object], exclude=[np.number])
test_df.drop(test_categ_df.columns, axis=1, inplace=True)

# One-hot encode
train_categ_df = pd.get_dummies(train_categ_df, dummy_na=True)
test_categ_df = pd.get_dummies(test_categ_df, dummy_na=True)
train_df = pd.concat([train_df, train_categ_df], axis=1)
test_df = pd.concat([test_df, test_categ_df], axis=1)
test_df.columns

Index(['age_r', 'yrsqual', 'yrsqual_t', 'yrsget', 'imyrs', 'leavedu',
       'nfehrsnjr', 'nfehrsjr', 'nfehrs', 'isco1c',
       ...
       'v127_ISCED 3A-B', 'v127_ISCED 3C 2 years or more',
       'v127_ISCED 3C shorter than 2 years',
       'v127_ISCED 4 (without distinction A-B-C)', 'v127_ISCED 4A-B',
       'v127_ISCED 5A, bachelor degree',
       'v127_ISCED 5A, master degree, and 6 (without distinction)',
       'v127_ISCED 5B', 'v127_No formal qualification or below ISCED 1',
       'v127_nan'],
      dtype='object', length=2531)

In [15]:
# Take the intersection of train_df and test_df columns
train_col_intersect = []
test_col_intersect = []
for col in train_df.columns:
    if col in test_df.columns:
        train_col_intersect.append(col)
for col in test_df.columns:
    if col in train_df.columns:
        test_col_intersect.append(col)
len(train_col_intersect) == len(test_col_intersect), len(train_col_intersect)

(True, 2299)

In [16]:
# get cols to drop
train_col_to_drop = set(train_df.columns) - set(train_col_intersect)
test_col_to_drop = set(test_df.columns) - set(test_col_intersect)
len(train_col_to_drop)

878

In [17]:
train_df.shape

(20000, 3177)

In [18]:
# drop
train_df.drop(train_col_to_drop, axis=1, inplace=True)
test_df.drop(test_col_to_drop, axis=1, inplace=True)

In [19]:
train_df.shape

(20000, 2299)

In [20]:
# Divide into X and Y (for submission)
X_train_final = train_df.drop('job_performance', axis=1) # Fit on train_final's
y_train_final = train_df['job_performance']
X_test_final = test_df.drop('job_performance', axis=1) # and predict on X_test_final for submission
y_test_final = test_df['job_performance'] # Empty

# Pipeline for Model Selection and Tuning
Splitting train_df into train and test because only train_df has true labels to check accuracy

In [21]:
from sklearn.model_selection import train_test_split

df = train_df[:]
X = train_df.drop('job_performance', axis=1)
y = train_df['job_performance']
print(X.shape, y.shape)

(20000, 2298) (20000,)


In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train.shape, y_test.shape

((14000, 2298), (6000,))

In [23]:
def evaluate_MSE(MSE):
    '''Evaluates MSE based on HW5 problem statement'''
    MSE = MSE * 20000/6000
    near_upper = 164100
    near_lower = 44900
    unit = 1950
    
    if MSE > near_upper:
        print(f'MSE of {MSE} exceeds the upper bound for 90% range')
    elif MSE > near_lower:
        print(f'MSE of {MSE} is within 90% range for grade')
    else:
        ec = int((near_lower - MSE) / unit)
        print(f'MSE of {MSE} qualifies for {ec}% of EC')

# Create Baseline Model
Basic OLS regression

In [24]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train, y_train)
reg.score(X_train, y_train)

0.721883546025893

In [25]:
y_true = y_test[:]
y_pred = reg.predict(X_test)
y_pred.shape

(6000,)

In [26]:
from sklearn.metrics import mean_squared_error
MSE = mean_squared_error(y_true, y_pred)
MSE

78050.50255177343

In [27]:
evaluate_MSE(MSE)

MSE of 260168.34183924476 exceeds the upper bound for 90% range


# Decision Tree Regressor

In [28]:
from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor(max_features='auto',random_state=0)
dtr.fit(X_train, y_train)
y_pred = dtr.predict(X_test)
MSE_dtr = mean_squared_error(y_true, y_pred)

print(dtr.score(X_train, y_train))
evaluate_MSE(MSE_dtr)

0.9999880938127264
MSE of 116503.72624054905 is within 90% range for grade


# Submission csv generation

In [29]:
# fit on all of trainset
dtr.fit(X_train_final, y_train_final)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features='auto',
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=0, splitter='best')

In [30]:
dtr.score(X_train_final, y_train_final)

0.9999864076619248

In [37]:
submission_pred = dtr.predict(X_test_final)
submission_pred

array([2846.33309595, 3159.7792572 , 3160.48745456, ..., 2567.88613056,
       2720.04949412, 3487.44581598])

In [39]:
submission_pred = pd.Series(submission_pred)
submission_pred.head()

0    2846.333096
1    3159.779257
2    3160.487455
3    3292.071202
4    2549.473597
dtype: float64

In [41]:
submission_pred.to_csv('hw5-testset-jw3580-submission.csv')

C:\Users\justi\Anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
